In [11]:
import pickle as pk
import torch
import pandas as pd

In [43]:
# load datasets and embeddings
mnli_train_dict = pk.load(open("./hw2_data/mnli_train_dict.pk", "rb"))
mnli_val_dict = pk.load(open("./hw2_data/mnli_val_dict.pk", "rb"))

snli_train_id = pk.load(open("./hw2_data/snli_train_id.pk", "rb"))
snli_val_id = pk.load(open("./hw2_data/snli_val_id.pk", "rb"))

loaded_embeddings_ft = pk.load(open("./hw2_data/loaded_embeddings_ft.pk", "rb"))
Genres = list(mnli_train_dict.keys())

In [41]:
from rnn_trainer import rnn_trainer

model_path = 'best_models/RNN.pth'
with open(model_path, 'rb') as model_dict:
        checkpoint = torch.load(model_dict)
best_args = checkpoint['config_dict']

In [44]:
mnli_rnn_acc = {}
for g in Genres:
    trainer = rnn_trainer(mnli_train_dict[g], mnli_val_dict[g], loaded_embeddings_ft, best_args)
    trainer.load_model('best_models', 'RNN')
    mnli_rnn_acc[g] = trainer.eval_stage()

trainer = rnn_trainer(snli_train_id, snli_val_id, loaded_embeddings_ft, best_args)
trainer.load_model('best_models', 'RNN')
mnli_rnn_acc['SNLI'] = trainer.eval_stage()


In [45]:
mnli_rnn_acc_df = pd.DataFrame.from_dict(mnli_rnn_acc, orient='index').T

In [48]:
from cnn_trainer import cnn_trainer
model_path = 'best_models/CNN.pth'
with open(model_path, 'rb') as model_dict:
        checkpoint = torch.load(model_dict)
best_args = checkpoint['config_dict']

In [51]:
mnli_cnn_acc = {}
for g in Genres:
    trainer = cnn_trainer(mnli_train_dict[g], mnli_val_dict[g], loaded_embeddings_ft, best_args)
    trainer.load_model('best_models', 'CNN')
    mnli_cnn_acc[g] = trainer.eval_stage()
    
trainer = cnn_trainer(snli_train_id, snli_val_id, loaded_embeddings_ft, best_args)
trainer.load_model('best_models', 'CNN')
mnli_cnn_acc['SNLI'] = trainer.eval_stage()


In [52]:
mnli_cnn_acc_df = pd.DataFrame.from_dict(mnli_cnn_acc, orient='index').T

In [53]:
MNLI_accuracy = pd.concat([mnli_cnn_acc_df, mnli_rnn_acc_df])
MNLI_accuracy.index = ['CNN', 'RNN']
MNLI_accuracy

,telephone,fiction,slate,government,travel,SNLI
CNN,46.666667,47.035176,44.311377,45.669291,46.945010,70.6
RNN,44.975124,48.241206,44.810379,45.374016,47.759674,70.0
